# ClimateHack.AI 2023 Data

Thank you for participating in ClimateHack.AI 2022! 

Your contributions could help cut carbon emissions by up to 100 kilotonnes per year in Great Britain alone. We look forward to seeing what you build over the course of the competition!

As with any machine learning task, the best place to start is by inspecting the data available, and for this competition, we are spoiled for choice!

## Prerequisites

If you do not have the following Python packages installed, you can uncomment and run the following line to install them with `pip`. 

In [ ]:
# %pip install numpy matplotlib zarr xarray ipykernel gcsfs fsspec dask cartopy ocf-blosc2

## Importing packages

In [ ]:
from datetime import datetime, time, timedelta

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from ocf_blosc2 import Blosc2

plt.rcParams["figure.figsize"] = (20, 12)

## HRV Satellite Imagery

In [ ]:
hrv = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-hrv/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

hrv

In [ ]:
hrv["data"].sel(time="2020-07-20 10:00").plot()  # type: ignore

In [ ]:
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=9.5))

hrv["data"].sel(time="2020-07-20 10:00", channel="HRV").plot.pcolormesh(
    ax=ax,
    transform=ccrs.Geostationary(central_longitude=9.5),
    x="x_geostationary",
    y="y_geostationary",
    add_colorbar=False,
)  # type: ignore

ax.coastlines()

## Non-HRV Satellite Imagery

In [ ]:
nonhrv = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-nonhrv/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

nonhrv

In [ ]:
nonhrv.channel

In [ ]:
nonhrv["data"].sel(time="2020-07-20 10:00", channel="IR_016").plot()  # type: ignore

In [ ]:
nwp = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/weather/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

nwp

### Ground temperatures

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

(nwp["t_g"].sel(time="2020-07-20 10:00") - 273.15).plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=True,
    cmap="coolwarm",
)  # type: ignore

ax.coastlines()

### Cloud cover

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

(nwp["clct"].sel(time="2020-07-20 10:00") - 273.15).plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=True,
)  # type: ignore

ax.coastlines()

## All weather variables

In [ ]:
nrows = 8
ncols = 5

fig, ax = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(10, 20),
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for i, var in enumerate(nwp.data_vars):
    nwp[var].sel(time="2020-07-20 10:00",).plot.pcolormesh(
        ax=ax[i // ncols][i % ncols],
        transform=ccrs.PlateCarree(),
        x="longitude",
        y="latitude",
        add_colorbar=False,
        cmap="coolwarm" if var.split("_")[0] in ("t", "v", "u") else "viridis",
    )

    ax[i // ncols][i % ncols].coastlines()
    ax[i // ncols][i % ncols].get_xaxis().set_visible(False)
    ax[i // ncols][i % ncols].get_yaxis().set_visible(False)
    ax[i // ncols][i % ncols].set_title(var)

fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)